In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
#定义模型
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.3018 - accuracy: 0.5903
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 680us/sample - loss: 1.2004 - accuracy: 0.6250 - val_loss: 0.7394 - val_accuracy: 0.7780
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4446 - accuracy: 0.8669
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 409us/sample - loss: 0.4322 - accuracy: 0.8750 - val_loss: 0.5279 - val_accuracy: 0.8390
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2762 - accuracy: 0.9250
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 481us/sample - loss: 0.2823 - accuracy: 0.9240 - val_loss: 0.4582 - val_accuracy: 0.8590
Epoch 4/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2059 - accuracy: 0.9587
Epoch

In [5]:
#新构建一个未经训练的全新模型，并用测试集对其进行评估。未训练模型的表现有很大的偶然性（准确率约为 10%）：
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 250us/sample - loss: 2.3964 - accuracy: 0.0750
Untrained model, accuracy:  7.50%


In [6]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 157us/sample - loss: 0.4043 - accuracy: 0.8720
Restored model, accuracy: 87.20%


## 检查点回调选项

该回调提供了多个选项，用于为生成的检查点提供独一无二的名称，以及调整检查点创建频率。

训练一个新模型，每隔 5 个周期保存一次检查点并设置唯一名称：

In [9]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)




Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 325us/sample - loss: 0.4848 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [12]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')


Epoch 1/5
1000/1000 [==============================] - 1s 578us/sample - loss: 1.1516 - accuracy: 0.6780
Epoch 2/5
1000/1000 [==============================] - 0s 352us/sample - loss: 0.4318 - accuracy: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 329us/sample - loss: 0.2845 - accuracy: 0.9290
Epoch 4/5
1000/1000 [==============================] - 0s 351us/sample - loss: 0.2122 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 310us/sample - loss: 0.1463 - accuracy: 0.9670


In [13]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 357us/sample - loss: 0.4091 - accuracy: 0.8630
Restored model, accuracy: 86.30%
